In [2]:
# allows editing aoc_utils "live" without restarting kernel
# see https://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
# and https://stackoverflow.com/a/17551284
%load_ext autoreload
%autoreload 2

# Add the aoc_utils path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import aoc_utils
get_input = aoc_utils.get_input
print = aoc_utils.debug_print

timer = aoc_utils.start_timer()

In [3]:
# Useful imports
import re
from collections import defaultdict, deque
import heapq
import functools
import queue
import itertools
import math


In [4]:
data = aoc_utils.get_input(1, 2015)[0]
level = 0
p2 = None
for idx,c in enumerate(data):
  if c == '(':
    level += 1
  elif c == ')':
    level -= 1
  if level == -1 and p2 is None:
    p2 = idx + 1
p1 = level
print(f"p1 {p1}, p2 {p2}")

p1 280, p2 1797


In [5]:
data = aoc_utils.get_input(2, 2015)
print = aoc_utils.debug_print
print = aoc_utils.noop
p1 = 0
p2 = 0
for line in data:
  l,w,h = aoc_utils.mapints(line)
  sides = [l*w, l*h, h*w]
  faces = [2*x for x in [l,w,h]]
  areas = [2*side for side in sides]
  area = sum(areas)
  vol = l*w*h
  p1 += area + min(sides)
  print(vol, faces, sorted(faces)[:2])
  p2 += vol + sum(sorted(faces)[:2])
p1,p2


(1586300, 3737498)

In [6]:
data = aoc_utils.get_input(3,2015)
G = defaultdict(lambda:0)
G2 = defaultdict(lambda:0)
G3 = defaultdict(lambda:0)
deltas = {
  '>': (1,0),
  '<': (-1,0),
  '^': (0,1),
  'v': (0,-1)
}
pos = (0,0)
G[pos] += 1
g2pos = (0,0)
g3pos = (0,0)
G2[g2pos] += 1
G2[g3pos] += 1
for line in data:
  for idx,ch in enumerate(line):
    delta = deltas[ch]
    pos = (pos[0] + delta[0], pos[1]+delta[1])
    G[pos] += 1
    if idx % 2 == 0:
      g2pos = (g2pos[0] + delta[0], g2pos[1]+delta[1])
      G2[g2pos] += 1
    else:
      g3pos = (g3pos[0] + delta[0], g3pos[1]+delta[1])
      G2[g3pos] += 1
p1 = len(G.keys())
p2 = len(G2.keys())
p1,p2



(2572, 2631)

In [94]:
data = aoc_utils.get_input(4, 2015)[0]
def check_num(prefix,num,len=5):
  return aoc_utils.md5(f"{prefix}{num}").startswith('0'*len)
p1 = 0
p2 = 0
while not check_num(data,p1):
  p1 += 1
while not check_num(data,p2,len=6):
  p2 += 1
p1,p2


(282749, 9962624)

In [95]:
from itertools import pairwise


print = aoc_utils.noop

data = aoc_utils.get_input(5,2015)
p1 = 0
p2 = 0
vowels = 'aeiou'
avoid = ['ab', 'cd', 'pq', 'xy']
def triplewise(iterable):
  for (a,b), (_,c) in pairwise(pairwise(iterable)):
    yield (a,b,c)
for line in data:
  vowel_count = len([ch for ch in line if ch in vowels])
  has_dupe = any(line[i] == line[i+1] for i in range(len(line)-1))
  has_avoid = any(x in line for x in avoid)
  if vowel_count >= 3 and has_dupe and not has_avoid:
    p1 += 1

  has_dupe_2 = any(line[idx:idx+2] in line[idx+2:] for idx in range(len(line)-2))
  has_trio = any(a==c for (a,b,c) in triplewise(line))
  if has_dupe_2 and has_trio:
    print(line)
    p2 += 1
  

p1,p2

(236, 51)

In [101]:
from aoc_utils import mapints


data = aoc_utils.get_input(6, 2015)

G = defaultdict(int)
G2 = defaultdict(int)
ON = lambda x:1
OFF = lambda x:0
TOGGLE = lambda x:1-x
ON2 = lambda x:x+1
OFF2 = lambda x:max(x-1,0)
TOGGLE2 = lambda x:x+2 
for line in data:
  inst = ON if 'on' in line else OFF if 'off' in line else TOGGLE
  inst2 = ON2 if 'on' in line else OFF2 if 'off' in line else TOGGLE2
  x1,y1,x2,y2 = mapints(line)
  assert(x1 <= x2 and y1 <= y2)
  for x in range(x1,x2+1):
    for y in range(y1,y2+1):
      G[(x,y)] = inst(G[(x,y)])
      G2[(x,y)] = inst2(G2[(x,y)])
p1 = sum(G.values())
p2 = sum(G2.values())
p1,p2


(569999, 17836115)

In [7]:
print = aoc_utils.debug_print
print = aoc_utils.noop
data = aoc_utils.get_input(7,2015)
data
G = {}

def isint(s):
  try:
    int(s)
    return True
  except ValueError:
    return False
def tryint(s):
  try:
    return int(s)
  except ValueError:
    return s

op_fns = {
  'AND': lambda x,y: x&y,
  'RSHIFT': lambda x,y: x >> y,
  'LSHIFT': lambda x,y: x << y,
  'OR': lambda x,y: x | y
}

def myeval(expr, G):
  print(expr)
  if isinstance(expr, int):
    return expr
  elif isinstance(expr, str):
    assert expr in G
    retval = myeval(G[expr], G)
    G[expr] = retval
    return retval
  else:
    op, *operands = expr
    if op is None:
      assert len(operands) == 1
      return myeval(operands[0], G)
    elif op == "NOT":
      assert len(operands) == 1
      return ~myeval(operands[0], G)
    else:
      assert(op in ['AND','RSHIFT','OR','LSHIFT'])
      assert(len(operands) == 2)
      fn = op_fns[op]
      lhs,rhs = operands
      return myeval( fn(myeval(lhs, G), myeval(rhs, G)), G )

for line in data:
  _in,wire = line.split(' -> ')
  assert(not isint(wire))
  if any([op in _in for op in ['AND','RSHIFT','OR', 'LSHIFT']]):
    lhs,op,rhs = _in.split(' ')
    G[wire] = (op, tryint(lhs),tryint(rhs))
  elif 'NOT' in _in:
    op, rhs = _in.split(' ')
    assert(op == "NOT")
    G[wire] = (op, tryint(rhs))
  else:
    G[wire] = (None, tryint(_in))

p1 = myeval('a', G)

G = {}

for line in data:
  _in,wire = line.split(' -> ')
  assert(not isint(wire))
  if any([op in _in for op in ['AND','RSHIFT','OR', 'LSHIFT']]):
    lhs,op,rhs = _in.split(' ')
    G[wire] = (op, tryint(lhs),tryint(rhs))
  elif 'NOT' in _in:
    op, rhs = _in.split(' ')
    assert(op == "NOT")
    G[wire] = (op, tryint(rhs))
  else:
    G[wire] = (None, tryint(_in))

G['b'] = p1
p2 = myeval('a', G)
p1,p2


(16076, 2797)

In [60]:
print = aoc_utils.debug_print
data = aoc_utils.get_input(8,2015)
def calc_str_len(s):
  l = 0
  idx = 0
  while idx < len(s):
    b = s[idx]
    if chr(b) == "\\":
      idx += 1
      next_b = s[idx]
      if chr(next_b) == "\\": # \\ -> \
        idx += 1
        l += 1
      elif chr(next_b) == '"': # \" -> "
        idx += 1
        l += 1
      elif chr(next_b) == "x": # \xab -> 1
        idx += 3
        l += 1
      else:
        print(b,next_b,idx,s)
        assert False
    else:
      idx += 1
      l += 1

  return l

def expand_s(s):
  return 2 + sum(expand_b(b) for b in s.encode())

def expand_b(b):
  if chr(b) == "\"":
    return 2
  elif chr(b) == "\\":
    return 2
  else:
    return 1


len_mem = 0
len_str = 0
for line in data:
  len_mem += len(line)
  len_str += calc_str_len( line[1:-1].encode() )

p1 = len_mem-len_str
p1

len_mem = 0
len_mem_expanded = 0
for line in data:
  len_mem += len(line)
  len_mem_expanded += expand_s(line)

p2 = len_mem_expanded-len_mem

p1,p2


(1371, 2117)

In [70]:
data = aoc_utils.get_input(9, 2015)
data

G = defaultdict(lambda: {})
for line in data:
  lhs,rhs = line.split(' to ')
  rhs,d = rhs.split(' = ')
  d = int(d)
  G[lhs][rhs] = d
  G[rhs][lhs] = d

def dist_tour(tour, G):
  return sum(G[a][b] for a,b in itertools.pairwise(tour))

cities = list(set(G.keys()))
p1 = min([dist_tour(tour, G) for tour in itertools.permutations(cities, len(cities))])
p2 = max([dist_tour(tour, G) for tour in itertools.permutations(cities, len(cities))])
p1,p2

(207, 804)

In [83]:
x = "1121"

def make_groups(x):
  groups = []
  group = (1, x[0])
  idx = 1
  while idx < len(x):
    if x[idx] == group[1]:
      group = (group[0] + 1, group[1])
    else:
      groups.append(group)
      group = (1, x[idx])
    idx += 1
  groups.append(group)
  return groups

def expand(s):
  groups = make_groups(s)
  return "".join(f"{group[0]}{group[1]}" for group in groups)

expand("111221")

data = aoc_utils.get_input(10, 2015)[0]

for idx in range(50):
  data = expand(data)
  if idx == 39:
    p1 = len(data)

p2 = len(data)
p1,p2



(329356, 4666278)

In [129]:
data = aoc_utils.get_input(11, 2015)[0]

import string
INVALID_CHARS = ['i','o','l']
INVALID_NUMS = [string.ascii_lowercase.find(c) for c in INVALID_CHARS]

def inc_enc_at(p, idx):
  max_v = len(string.ascii_lowercase) - 1
  if p[idx] == max_v:
    return inc_enc_at(p, idx - 1)
  return p[:idx] + [p[idx] + 1] + [0] * (len(p) - idx - 1)

def inc_enc(p):
  try:
    bad_idx = next(idx for idx,v in enumerate(p) if v in INVALID_NUMS)
    return inc_enc_at(p, bad_idx)
  except StopIteration:
    return inc_enc_at(p, len(p) - 1)

import itertools
pairwise = itertools.pairwise

def triplewise(iter):
  import itertools
  pairwise = itertools.pairwise
  for (a,_),(b,c) in pairwise(pairwise(iter)):
    yield (a,b,c)

def is_valid_enc(p):
  has_invalid_num = any(n in p for n in INVALID_NUMS)
  if has_invalid_num: return False
  has_3 = any( a + 1 == b and b + 1 == c for (a,b,c) in triplewise(p))
  if not has_3: return False
  has_2 = len(set((a,b) for (a,b) in pairwise(p) if a == b)) > 1
  return has_2 and has_3 and not has_invalid_num


def encode_pass(p):
  import string
  return [string.ascii_lowercase.index(c) for c in p]

def decode_pass(p):
  import string
  return ''.join([string.ascii_lowercase[idx] for idx in p])

assert decode_pass(encode_pass(data)) == data
assert is_valid_enc(encode_pass('abcdffaa'))
assert is_valid_enc(encode_pass('ghjaabcc'))
assert not is_valid_enc(encode_pass('hijklmmn'))
assert not is_valid_enc(encode_pass('abbceffg'))
assert not is_valid_enc(encode_pass('abbcegjk'))

p1 = None
p  = encode_pass(data)
for _ in itertools.count():
  if is_valid_enc(p):
    p1 = decode_pass(p)
    break
  p = inc_enc(p)

p = inc_enc(p)
for _ in itertools.count():
  if is_valid_enc(p):
    p2 = decode_pass(p)
    break
  p = inc_enc(p)

p1,p2



('vzbxxyzz', 'vzcaabcc')

In [140]:
data = aoc_utils.get_input(12, 2015)[0]

p1 = sum(aoc_utils.mapints(data))
p1

def sum_list(arr):
  return sum(sum_item(i) for i in arr)

def sum_dict(obj):
  if any(v == "red" for v in obj.values()): return 0
  return sum(sum_item(v) for v in obj.values())

def sum_item(i):
  if isinstance(i, list):
    return sum_list(i)
  elif isinstance(i, dict):
    return sum_dict(i)
  elif isinstance(i, int):
    return i
  elif isinstance(i, str):
    return 0
  else:
    print(i)
    assert False

import json
p2 = sum_item(json.loads(data))

p1,p2

(119433, 68466)

In [147]:
data =aoc_utils.get_input(13, 2015)
data

G = defaultdict(dict)
ppl = set()
for line in data:
  lhs,_ = line.split(' would ')
  _,rhs = line.split('next to ')
  rhs,_ = rhs.split('.')
  v = aoc_utils.mapints(line)[0]
  if 'lose' in line: v *= -1
  ppl.add(lhs)
  ppl.add(rhs)
  G[lhs][rhs] = v

from itertools import pairwise, permutations

totals = []
for perm in permutations(ppl):
  totals.append(
    sum(G[lhs][rhs] + G[rhs][lhs] for lhs,rhs in pairwise(perm)) +
    G[perm[-1]][perm[0]] +
    G[perm[0]][perm[-1]]
  )

p1 = max(totals)

for person in ppl:
  G[person]['me'] = 0
  G['me'][person] = 0

ppl.add('me')
totals = []
for perm in permutations(ppl):
  totals.append(
    sum(G[lhs][rhs] + G[rhs][lhs] for lhs,rhs in pairwise(perm)) +
    G[perm[-1]][perm[0]] +
    G[perm[0]][perm[-1]]
  )
p2 = max(totals)

p1,p2

(709, 668)

In [161]:
data = aoc_utils.get_input(14, 2015)
data

init = {}
state = {}

for line in data:
  v,vt,rt = aoc_utils.mapints(line)
  r,*_ = line.split(' ')
  init[r] = (0,v,vt,rt)
  state[r] = (0,v,vt,rt,0) # distance, velocity, velocity_time, rest_time, pts

def tick(state):
  next_state = {}
  for r,(d,v,vt,rt,pts) in state.items():
    if vt > 0:
      vt -= 1
      d += v
      if vt == 0:
        rt = init[r][3]
    else:
      assert vt == 0
      if rt > 0:
        rt -= 1
      if rt == 0:
        vt = init[r][2]
    next_state[r] = (d,v,vt,rt,pts)
  leader_d = max(d for (d,v,vt,rt,pts) in next_state.values())
  leader_rs = []
  for r,(d,v,vt,rt,pts) in next_state.items():
    if d == leader_d:
      leader_rs.append(r)
  assert len(leader_rs) > 0
  for leader_r in leader_rs:
    (d,v,vt,rt,pts) = next_state[leader_r]
    next_state[leader_r] = (d,v,vt,rt,pts+1)

  return next_state

for _ in range(2503):
  state = tick(state)

p1 = max(d for (d,*_) in state.values())
p2 = max(pts for (d,v,vt,rt,pts) in state.values())

p1,p2




(2640, 1102)

In [132]:
from itertools import combinations, combinations_with_replacement, product
from functools import cache


data = aoc_utils.get_input(15, 2015)
data


data = """Butterscotch: capacity -1, durability -2, flavor 6, texture 3, calories 8
Cinnamon: capacity 2, durability 3, flavor -2, texture -1, calories 3""".split('\n')

data = aoc_utils.get_input(15, 2015)

print = aoc_utils.debug_print

ings = {}
prop_names = set()
ing_names = set()
for line in data:
  name,rest = line.split(': ')
  props = rest.split(', ')
  props = [prop.split(' ') for prop in props]
  ings[name] = {}
  for prop,val in props:
    ings[name][prop] = int(val)

for ing_name,props in ings.items():  
  ing_names.add(ing_name)
  for prop_name,val in props.items():
    prop_names.add(prop_name)
prop_names = list(prop_names)
ing_names = list(ing_names)

@cache
def score_cookie2(amts):
  total = 1
  for prop_name in prop_names:
    if prop_name == 'calories': continue
    subtotal = score_cookie_prop2(prop_name, amts)
    if subtotal == 0: return 0
    total *= subtotal
  return total

@cache
def score_cookie_prop2(prop_name, amts):
  subtotal = 0
  prop_vals = [ings[name][prop_name] for name in ing_names]
  subtotal = sum([amts[idx]*prop_vals[idx] for idx in range(len(amts))])
  if subtotal <= 0: return 0
  return subtotal

def find_combos(target_sum=100, n=4):
  from itertools import combinations_with_replacement,permutations
  for tuple in combinations_with_replacement(range(target_sum+1), n):
    if sum(tuple) == target_sum:
      for tuple in permutations(tuple, len(tuple)):
        yield tuple

p1 = max(score_cookie2(combo) for combo in find_combos(100,len(ing_names)))
p2 = max(score_cookie2(combo) for combo in find_combos(100,len(ing_names)) if score_cookie_prop2('calories', combo) == 500)

p1,p2

(13882464, 11171160)

In [15]:
data = aoc_utils.get_input(16, 2015)
data

known = {
  "children": 3,
  "cats": 7,
  "samoyeds": 2,
  "pomeranians": 3,
  "akitas": 0,
  "vizslas": 0,
  "goldfish": 5,
  "trees": 3,
  "cars": 2,
  "perfumes": 1
}

sues = defaultdict(list)
for line in data:
  idx = aoc_utils.mapints(line)[0]
  props = line[line.index(": ") + 2:]
  props = props.split(', ')
  sues[idx] = {
    prop.split(': ')[0]: int(prop.split(': ')[1])
    for prop in props
  }


p1 = 0
p2 = 0
for sue,props in sues.items():
  if all([known[name] == val for name,val in props.items()]):
    p1 = sue

def find_sue2(props):
  for name,val in props.items():
    if name in ["cats", "trees"] and val > known[name]: continue
    if name in ["pomeranians", "goldfish"] and val < known[name]: continue
    if known[name] == val: continue
    return False
  return True

for sue, props in sues.items():
  if sue == p1: continue
  if find_sue2(props):
    p2 = sue


p1,p2

(373, 260)

In [34]:
data = aoc_utils.get_input(17, 2015)
data = list(map(int, data))
data = list(sorted(data))
sum(data[:9])
sum(data[-4:])
p1 = 0
ways = defaultdict(int)
for i in range(4,10):
  for combo in itertools.combinations(data, i):
    if sum(combo) == 150:
      ways[len(combo)] += 1

p1 = sum(ways.values())
p2 = ways[min(ways.keys())]
p1,p2

(654, 57)

In [43]:
data = aoc_utils.get_input(18, 2015)
data

rowmax = len(data)-1
colmax = len(data[0])-1
corners = [
  (0,0),
  (0,rowmax),
  (colmax, 0),
  (colmax,rowmax)
]

def get_grid(data,is_p2=False):
  G = {}
  for row,line in enumerate(data):
    for col,ch in enumerate(line):
      G[(col,row)] = 1 if ch == '#' else 0
  if is_p2:
    for p in corners: G[p] = 1
  return G

def tick(G, is_p2=False):
  G2 = {}
  for p,val in G.items():
    neighbors = [n for n in aoc_utils.neighbors8( p ) if n in G]
    count = sum([G[n] for n in neighbors])
    if val == 1:
      G2[p] = 1 if count in [2,3] else 0
    elif val == 0:
      G2[p] = 1 if count == 3 else 0
    else:
      assert False
  if is_p2:
    for p in corners: G2[p] = 1

  return G2

G = get_grid(data)
for _ in range(100):
  G = tick(G)
p1 = sum(G.values())

G2 = get_grid(data, is_p2=True)
for _ in range(100):
  G2 = tick(G2, is_p2=True)
p2 = sum(G2.values())

p1,p2


(1061, 1006)